In [121]:
def toLower(s):
    return s.replace(' ', '').lower()

def appPotrebEncode(x):
    if x == 0:
        return 0
    return 1

def toZeroOne(x):
    if x <= 0:
        return 0
    return 1

In [146]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

path = './data/'
filenameA = 'aggregates'
filenameT = 'transactions'

df = pd.read_csv(path + filenameA + '.csv')
#app_potreb_cnt
set( df.report_date.tolist() )

{'2019-01-01 00:00:00',
 '2019-02-01 00:00:00',
 '2019-03-01',
 '2019-03-01 00:00:00',
 '2019-04-01 00:00:00',
 '2019-05-01 00:00:00',
 '2019-06-01 00:00:00',
 '2019-07-01',
 '2019-07-01 00:00:00',
 '2019-08-01',
 '2019-08-01 00:00:00',
 '2019-09-01',
 '2019-09-01 00:00:00',
 '2019-10-01',
 '2019-10-01 00:00:00',
 '2019-11-01',
 '2019-11-01 00:00:00',
 '2019-12-01',
 '2019-12-01 00:00:00',
 '2020-01-01',
 '2020-01-01 00:00:00',
 '2020-02-01',
 '2020-02-01 00:00:00',
 '2020-03-01',
 '2020-03-01 00:00:00',
 '2020-04-01',
 '2020-04-01 00:00:00',
 '2020-05-01',
 '2020-05-01 00:00:00',
 '2020-06-01',
 '2020-06-01 00:00:00',
 '2020-07-01',
 '2020-07-01 00:00:00',
 '2020-08-01',
 '2020-08-01 00:00:00',
 '2020-09-01',
 '2020-09-01 00:00:00',
 '2020-10-01',
 '2020-10-01 00:00:00'}

In [ ]:
df = df.drop(df[df.app_potreb_cnt.isnull()].index)

to_drop = []
df.drop(to_drop, axis=1, inplace=True)
# If more then 30% info is nan
df = df.loc[:, (df.isnull().sum(axis=0)/len(df.app_potreb_cnt) <= 0.3)]

In [ ]:
#Transforming Data
df.report_date = dates = pd.to_datetime(df.report_date).astype(int) / 10**9
df.head()

In [ ]:
#GENERATING NEW DATAFRAME
data = pd.DataFrame()

df_grouped = df.groupby('client_id')['report_date'].apply(list)
df_grouped = df_grouped.reset_index()

for client_id in set( df_grouped.client_id.tolist() ):
    report_date_list = df_grouped.loc[df_grouped.client_id == client_id].report_date.item()
    report_date_list.sort()
    for i in range(1, len(report_date_list)):
        a_row = df.loc[(df.client_id == client_id) & (df.report_date == report_date_list[i-1])]
        data_new_row = pd.DataFrame(a_row)

        y0 = a_row.app_potreb_cnt.item()
        y1 = df.loc[(df.client_id == client_id) & (df.report_date == report_date_list[i])].app_potreb_cnt.item()

        data_new_row['y'] = [y1-y0]

        data = pd.concat([data_new_row, data], ignore_index=True)

data.head()

In [ ]:
#Encoding
for p in data.select_dtypes(include=['object']):
    data[p] = [toLower(str(x)) for x in data[p]]
    
for p in data.select_dtypes(include=['object']):
    names = list(set(data[p]))
    data[p] = [names.index(x) for x in data[p]]

data.y = [toZeroOne(x) for x in data.y]

In [130]:
data.head()

,client_id,report_date,cnt_zp_1m,cnt_zp_3m,cnt_zp_6m,cnt_zp_12m,total_num_cards,cnt_active,cnt_cur,first_opened,...,curdel_60plus_bank,curdel_60plus_bki,curdel_tech_bank,curdel_tech_bki,bank_1_29_5y_debtor,bank_30_59_5y_debtor,bank_60_89_5y_debtor,bank_90_119_5y_debtor,bank_120plus_5y_debtor,y
0,6742,1.559347e+09,2.0,10.0,18.0,34.0,2.0,1.0,0.0,65.258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6742,1.556669e+09,5.0,12.0,20.0,35.0,2.0,1.0,0.0,64.258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,6742,1.554077e+09,3.0,8.0,17.0,32.0,2.0,1.0,0.0,63.258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6741,1.598918e+09,3.0,8.0,16.0,32.0,13.0,2.0,0.0,130.774,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0
4,6741,1.596240e+09,3.0,8.0,17.0,32.0,13.0,2.0,0.0,129.774,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0


In [131]:
#РАЗВЕДОЧНЫЙ АНАЛИЗ ДАННЫХ heatmap Корреляций
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

In [143]:
for cl in data.columns:
    data[cl].fillna(data[cl].mean())

0.019143104846603462

In [127]:
data.to_csv(path + filenameA + 'newData.csv')